## tsvファイルを読み込んでBERTserver経由でテンソルに変換

後で文をトークナイズするモデルを使うため、事前に文ごとにEmbeddingをかけておく

In [3]:
#BERTを使って文ベクトルに変換
from bert_serving.client import BertClient
server_ip = '172.16.16.171' #BERTserverのipアドレス
bc_jp = BertClient(ip=server_ip,port=5555,port_out=5556,check_length=False,check_version=False)

import sentencepiece as spm
import numpy as np
import torch
import re

s = spm.SentencePieceProcessor()
s.Load("./wiki-ja.model")

def parse(text):
    text = text.lower()
    return s.EncodeAsPieces(text)

#tensorを返す関数を定義
def sentence2vec(text):
    sentence_lists = []
    sentence_lists = re.split('[。!]',text) #「。」と「！」で文を分割
    if len(sentence_lists) > 1:
        del sentence_lists[-1] #一番最後はNoneになるので消しておく
    
    parsed_texts = list(map(parse,sentence_lists))
    parsed_texts = [txt for txt in parsed_texts if (txt != [] and txt != '')]
    array = bc_jp.encode(parsed_texts,is_tokenized = True)
    return array.tolist() #すべての文の分散表現を並べたリストを返す。

先ほど作ったtsvを読み込んできて、テンソルに変換する関数を定義

In [16]:
import pandas as pd
from tqdm import tqdm

def doc2tensor(text): #引数でデータの種類を選択

    filepath = ''
    if text == 'train':
        filepath += './train_livedoor.tsv'
    elif text == 'val':
        filepath += './val_livedoor.tsv'
    elif text == 'test':
        filepath += './test_livedoor.tsv'
        
    origin_df = pd.read_csv(filepath,sep='\t')
    origin_df = origin_df.sample(frac=1).reset_index(drop=True)

    df = pd.DataFrame(columns=['main_doc','label'],index=range(len(origin_df)))

    for i in tqdm(range(len(origin_df))):
        df.iloc[i,:] = [sentence2vec(origin_df['main_doc'][i]),origin_df['label'][i]]
    
    return df

train,val,testをそれぞれEmbeddingかけた状態でpickle保存する

In [18]:
import pickle

tmp = ['train','val','test']

for i in range(len(tmp)):
    kind = tmp[i]
    with open('./embedded_{}_livedoor.pickle'.format(kind),'wb')as f:
        df = doc2tensor(kind)
        pickle.dump(df,f)


100%|██████████| 5000/5000 [1:03:24<00:00,  1.31it/s]

100%|██████████| 1000/1000 [14:34<00:00,  1.14it/s]

100%|██████████| 1000/1000 [14:59<00:00,  1.11it/s]


中身はこんな感じ

In [19]:
df.head()

,main_doc,label
0,"[[0.22710075974464417, -0.30410313606262207, 0...",8
1,"[[0.6103750467300415, -0.5876586437225342, -0....",2
2,"[[-0.23334041237831116, 0.1732082962989807, 0....",6
3,"[[0.21237729489803314, -0.5661413669586182, 0....",1
4,"[[0.044950246810913086, 0.3979344964027405, 0....",7
